# Linear Optimization Methods

we have algorithm to select relevant layouts. It is the integer knapsack problem

no we just need to put it all together

In [1]:
import sys
sys.path.append("../")

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import math
from random import shuffle, choice
import copy
from utils import *
from engine import *
from IPython.display import clear_output
import time
import datetime
import re

In [2]:
df = pd.read_excel('../data/200721_ New Way SCH W3-W6 W14 07.20.20.xlsx',
                   sheet_name='Schedule')

# the input schedule appears as a data table on the dashboard
# this is used to select what to deckle
df_input_schedule = pd.read_excel('../data/200721_ New Way SCH W3-W6 W14 07.20.20.xlsx',
                   sheet_name='Schedule')
df_input_schedule.insert(1, 'Technology', df_input_schedule['Description'].apply(lambda x: parse_description(x, 'tech')))
df_input_schedule.insert(2, 'Color', df_input_schedule['Description'].apply(lambda x: x.split(';')[1] if type(x) == str else None))
df_input_schedule.insert(3, 'Width', df_input_schedule['Description'].apply(lambda x: parse_description(x, 'width')))
df_input_schedule = df_input_schedule[[col for col in df_input_schedule.columns if 'Unnamed' not in str(col)]]
df_input_schedule['Total LM Order QTY'] = df_input_schedule['Total LM Order QTY'].round(1)

In [3]:
customer = 'AHP'
technology = 'SM' # Technology -> 
color = 'WHITE'
cycle = 'CYCLE 1'

df_filtered = df.loc[df['Customer Name'] == customer]
df_filtered = df_filtered.loc[df_filtered['Description'].str.contains(technology)]
df_filtered = df_filtered.loc[df_filtered['Description'].str.contains(color)] #CYAN, TEAL
df_filtered = df_filtered.loc[df_filtered['CYCLE / BUCKET'] == cycle]
df_filtered = df_filtered.loc[df_filtered['Total LM Order QTY'] > 0]
df_filtered.insert(0, 'Block', 1)
df_filtered = df_filtered.reset_index(drop=True)
df_filtered['Order Number'] = df_filtered.index + 1

df_filtered['Width'] = pd.DataFrame(list(pd.DataFrame(list(df_filtered['Description']
                                                  .str.split(';')))[0].str.split('UN0')))[1]
df_filtered.head(10)

,Block,SPM375014 PROD. DATE,CYCLE / BUCKET,Customer Name,Item No,WO#,Description,Treatment,KG,Hours To Produce,...,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,% reclaim,Amount Repro needed,Type of Repro,Order Number,Width
0,1,2020-07-29 04:38:41.042,CYCLE 1,AHP,1559695,NaN,"SM30135UN0160;WHITE;600;46"" OD",UNTREATED,2000.0,1.131222,...,NaN,NaN,NaN,NaN,NaN,0.15,14451.705882,SMS,1,160
1,1,2020-07-29 05:46:33.441,CYCLE 1,AHP,1559618,NaN,"SM30135UN0195;WHITE;600;46"" OD",UNTREATED,7500.0,4.242081,...,NaN,NaN,NaN,NaN,NaN,0.15,14451.705882,SMS,2,195
2,1,2020-07-29 10:01:04.934,CYCLE 1,AHP,1715784,NaN,"SM30135UN0205;WHITE;600;46"" OD",UNTREATED,3000.0,1.696833,...,NaN,NaN,NaN,NaN,NaN,0.15,14451.705882,SMS,3,205
3,1,2020-07-29 11:42:53.531,CYCLE 1,AHP,1716031,NaN,"SM30135UN0220;WHITE;600;46"" OD",UNTREATED,14000.0,7.918552,...,NaN,NaN,NaN,NaN,NaN,0.15,14451.705882,SMS,4,220
4,1,2020-07-29 19:38:00.318,CYCLE 1,AHP,1559618,NaN,"SM30135UN0195;WHITE;600;46"" OD",UNTREATED,4725.0,2.672511,...,NaN,NaN,NaN,NaN,NaN,0.15,14451.705882,SMS,5,195
5,1,2020-07-29 22:18:21.359,CYCLE 1,AHP,1715784,NaN,"SM30135UN0205;WHITE;600;46"" OD",UNTREATED,7425.0,4.199661,...,NaN,NaN,NaN,NaN,NaN,0.15,14451.705882,SMS,6,205
6,1,2020-07-30 02:30:20.137,CYCLE 1,AHP,1716031,NaN,"SM30135UN0220;WHITE;600;46"" OD",UNTREATED,3037.0,1.717760,...,NaN,NaN,NaN,NaN,NaN,0.15,14451.705882,SMS,7,220
7,1,2020-07-30 04:13:24.074,CYCLE 1,AHP,1559695,NaN,"SM30135UN0160;WHITE;600;46"" OD",UNTREATED,5400.0,3.054299,...,NaN,NaN,NaN,NaN,NaN,0.15,14451.705882,SMS,8,160
8,1,2020-07-30 07:16:39.549,CYCLE 1,AHP,1559618,NaN,"SM30135UN0195;WHITE;600;46"" OD",UNTREATED,8100.0,4.581448,...,NaN,NaN,NaN,NaN,NaN,0.15,14451.705882,SMS,9,195
9,1,2020-07-30 11:51:32.762,CYCLE 1,AHP,1715784,NaN,"SM30135UN0205;WHITE;600;46"" OD",UNTREATED,8100.0,4.581448,...,NaN,NaN,NaN,NaN,NaN,0.15,14451.705882,SMS,10,205


In [4]:
lm = list(df_filtered.groupby('Description')['Total LM Order QTY'].sum().values)
widths = list(df_filtered.groupby('Width')['Total LM Order QTY'].sum().index.values.astype(int))
doffs_in_jumbo = 6
B = 4160
L = doff_length = 17000 * doffs_in_jumbo # df['LM putup']

# neckin = [4, 4, 5, 7, 7, 7] # 158 missing in cycle 1, 4 mm knife in
neckins = []
for width in widths:
    if width < 170:
        neckin = 4
    elif width < 208:
        neckin = 5
    else:
        neckin = 7
    neckins.append(neckin)
w = list(np.array(widths) + np.array(neckins)) # the values used in the actual computation
q = [math.ceil(x/L) for x in lm] # jumbos needed per width, rounded up
s = BinPackingExample(w, q) # material orders (list of widths, 1 width = 1 jumbo)

print('The important variables', end='\n\n')
print('widths: {} (mm)'.format(widths))
print('neck in: {} (mm)'.format(neckins))
print('L (m): {}'.format(L))
print('undeckled jumbos needed: {}'.format(q))

The important variables

widths: [160, 195, 205, 220, 235] (mm)
neck in: [4, 5, 5, 7, 7] (mm)
L (m): 102000
undeckled jumbos needed: [54, 344, 196, 212, 17]


In [5]:
def init_layouts(B, w):
    t = []
    m = len(w)
    for i in range(m):
        pat = [0]*m
        pat[i] = -int(B/w[i])
        t.append(pat)
    return t

In [6]:
init_layouts(B, w)

[[-25, 0, 0, 0, 0],
 [0, -20, 0, 0, 0],
 [0, 0, -19, 0, 0],
 [0, 0, 0, -18, 0],
 [0, 0, 0, 0, -17]]

In [7]:
from scipy.optimize import linprog

In [8]:
obj = [1,1,1,1,1]
lhs_ineq = init_layouts(B, w)
rhs_ineq = [-i for i in q]
print(rhs_ineq)
lhs_ineq

[-54, -344, -196, -212, -17]


[[-25, 0, 0, 0, 0],
 [0, -20, 0, 0, 0],
 [0, 0, -19, 0, 0],
 [0, 0, 0, -18, 0],
 [0, 0, 0, 0, -17]]

In [9]:
linprog(c=obj, 
        A_ub=lhs_ineq,
        b_ub=rhs_ineq, 
        method="revised simplex")

     con: array([], dtype=float64)
     fun: 42.45356725146199
 message: 'The solution was determined in presolve as there are no non-trivial constraints.'
     nit: 0
   slack: array([0., 0., 0., 0., 0.])
  status: 0
 success: True
       x: array([ 2.16      , 17.2       , 10.31578947, 11.77777778,  1.        ])

In [10]:
s = BinPackingExample(w, q)

# Linear Programming

![](https://files.realpython.com/media/lp-py-eq-3.65b2e6d529bc.png)

In [11]:
from scipy.optimize import linprog

In [12]:
obj = [-1, -2]
lhs_ineq = [[2, 1],
            [-4, 5],
            [1, -2]]

rhs_ineq = [20, 
            10, 
            2]

lhs_eq = [[-1, 5]]

rhs_eq = [15]

bnd = [(0, float("inf")), #bounds of x
       (0, float("inf"))] # bounds of y

In [13]:
linprog(c=obj, 
        A_ub=lhs_ineq,
        b_ub=rhs_ineq, 
        A_eq=lhs_eq,
        b_eq=rhs_eq,
        bounds=bnd,
        method="revised simplex")

     con: array([0.])
     fun: -16.818181818181817
 message: 'Optimization terminated successfully.'
     nit: 3
   slack: array([ 0.        , 18.18181818,  3.36363636])
  status: 0
 success: True
       x: array([7.72727273, 4.54545455])

## Stock Cutting

![](https://scipbook.readthedocs.io/en/latest/_images/csp-sol-a.png)

In [14]:
s = [6, 6, 5, 5, 5, 4, 4, 4, 4, 2, 2, 2, 2, 3, 3, 7, 7, 5, 5, 8, 8, 4, 4, 5]
w = [2,3,4,5,6,7,8]
B = 9

In [15]:
from collections import Counter

In [16]:
Counter(s)

Counter({6: 2, 5: 6, 4: 6, 2: 4, 3: 2, 7: 2, 8: 2})

In [17]:
# our layouts 
obj = [1,1,1,1,1,1,1]

# columns are layouts, rows are products
lhs_ineq = [[-4,0,0,0,0,0,0],
            [0,-2,0,0,0,0,0],
            [0,0,-2,0,0,0,0],
            [0,0,0,-1,0,0,0],
            [0,0,0,0,-1,0,0],
            [0,0,0,0,0,-1,0],
            [0,0,0,0,0,0,-1]]

rhs_ineq = [-4, 
            -2, 
            -6,
            -6,
            -2,
            -2, 
            -2]

bnd = [(0, float("inf")), #bounds of x
       (0, float("inf"))] # bounds of y

In [18]:
linprog(c=obj, 
        A_ub=lhs_ineq,
        b_ub=rhs_ineq, 
        method="revised simplex")

     con: array([], dtype=float64)
     fun: 17.0
 message: 'The solution was determined in presolve as there are no non-trivial constraints.'
     nit: 0
   slack: array([0., 0., 0., 0., 0., 0., 0.])
  status: 0
 success: True
       x: array([1., 1., 3., 6., 2., 2., 2.])

## Integer Knapsack Problem

In [19]:
# This is the memoization approach of  
# 0 / 1 Knapsack in Python in simple  
# we can say recursion + memoization = DP 
def knapsack(wt, val, W, n):     
  
    # base conditions 
    if n == 0 or W == 0: 
        return 0
    if t[n][W] != -1: 
        return t[n][W] 
  
    # choice diagram code 
    if wt[n-1] <= W: 
        t[n][W] = max( 
            val[n-1] + knapsack( 
            wt, val, W-wt[n-1], n-1),  
            knapsack(wt, val, W, n-1)) 
        return t[n][W] 
    elif wt[n-1] > W: 
        t[n][W] = knapsack(wt, val, W, n-1) 
        return t[n][W] 

def reconstruct(i, w, kp_soln, weight_of_item):
    """
    Reconstruct subset of items i with weights w. The two inputs
    i and w are taken at the point of optimality in the knapsack soln

    In this case I just assume that i is some number from a range
    0,1,2,...n
    """
    recon = set()
    # assuming our kp soln converged, we stopped at the ith item, so
    # start here and work our way backwards through all the items in
    # the list of kp solns. If an item was deemed optimal by kp, then
    # put it in our bag, otherwise skip it.
    for j in range(i)[::-1]:
        cur_val = kp_soln[j][w]
        prev_val = kp_soln[j-1][w]
        if cur_val > prev_val:
            recon.add(j)
            w = w - weight_of_item[j]
    return recon

def initt(W, n):
    # We initialize the matrix with -1 at first. 
    return [[-1 for i in range(W + 1)] for j in range(n + 1)] 

In [20]:
# driver code 
val = [60, 100, 120] 
wt = [1,1,1] 
W = 2
n = len(val) 
t = initt(W, n)
knapsack(wt, val, W, n)

220

In [21]:
t = np.array(t)
n, w = np.unravel_index(t.argmax(), t.shape) # returns n, W
print(n,w)
sack = reconstruct(n, w, t, wt)
np.array(val)[list(sack)]

3 2


array([100, 120])

In [22]:
s = [6, 6, 5, 5, 5, 4, 4, 4, 4, 2, 2, 2, 2, 3, 3, 7, 7, 5, 5, 8, 8, 4, 4, 5]
B = 9
t = initt(B, len(s))
knapsack(s, s, B, len(s))

9

In [23]:
t = np.array(t)
n, w = np.unravel_index(t.argmax(), t.shape) # returns n, W
print(n,w)

6 9


In [24]:
for pair in np.argwhere(t == t.max()):
    n, w = pair
    sack = reconstruct(n, w, t, s)
    print(sack)

{1}
{3, 6}
{3, 6}
{3, 6}
{3, 6}
{3, 6}
{3, 6}
{3, 6}
{3, 6}
{3, 6}
{3, 6}
{3, 6}
{3, 6}
{3, 6}
{3, 6}
{3, 6}
{3, 6}
{3, 6}
{3, 6}


In [25]:
sack = reconstruct(7, w, t, s)
sack

{3, 6}

In [26]:
np.array(s)[list(sack)]

array([5, 4])

In [27]:
B = 4160
w = list(np.array(widths) + np.array(neckins)) # the values used in the actual computation
q = [math.ceil(x/L) for x in lm] # jumbos needed per width, rounded up
s = BinPackingExample(w, q) # material orders (list of widths, 1 width = 1 jumbo)
t = [[-1 for i in range(B + 1)] for j in range(len(s) + 1)] 

knapsack(s, s, B, len(s))

4160

In [28]:
t = np.array(t)

In [29]:
n, w = np.unravel_index(t.argmax(), t.shape) # returns n, W
print(n,w)

410 4160


In [30]:
sack = reconstruct(n, w, t, s)

In [31]:
Counter(np.array(s)[list(sack)])

Counter({164: 7, 210: 1, 200: 14})

In [32]:
np.array(s)[list(sack)].sum()

4158

In [36]:
for bit in range(0,32*2,2):
    found = 0
    for pair in np.argwhere(t == t.max()-bit):
        n, w = pair
        sack = reconstruct(n, w, t, s)
        print(n,w)
        print(Counter(np.array(s)[list(sack)]))
        print(np.array(s)[list(sack)].sum(), end='\n\n')
        found += 1
        if found > 1:
            break

410 4160
Counter({200: 14, 164: 7, 210: 1})
4158

411 4160
Counter({210: 12, 164: 10})
4160

399 4160
Counter({164: 18, 200: 6})
4152

400 4160
Counter({200: 14, 164: 7, 210: 1})
4158

60 4160
Counter({164: 24, 200: 1})
4136

61 4160
Counter({164: 18, 200: 6})
4152

55 4160
Counter({164: 25})
4100

56 4160
Counter({164: 24, 200: 1})
4136

25 4160
Counter({164: 24})
3936

26 4160
Counter({164: 25})
4100



In [35]:
210*12+164*10

4160

# Knapsack with Cutting Stock